In [74]:
import pandas as pd
import importlib
import funciones as f
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, recall_score
import ast

importlib.reload(f)

<module 'funciones' from 'c:\\Users\\agutm\\OneDrive\\Escritorio\\Estudio\\Sistemasss\\SEVILLA\\InteligenciaArtificial\\Trabajo\\Trabajo\\funciones.py'>

In [2]:
import numpy as np
np.random.seed(357823) # Establecemos la semilla aleatoria inicial, para que el cuaderno sea reproducible

# ***Implementación de los algoritmos de búsqueda***

In [151]:
def BusquedaSecuencialHaciaAtras(atributos, objetivo, algoritmo, n_exp, k):
    solucionActual = np.array(atributos.columns[:].tolist())
    vectorMejoresSolucionesTemporales=[]

    for k in range(len(solucionActual), 1, -1):
        mejorRendimiento=-1;
        mejorSolucionTemporal = []
        for v in range(len(solucionActual)):
            solucionTemporal = solucionActual.copy();
            solucionTemporal = np.delete(solucionTemporal,v);
            rendimiento = f.EvaluacionRobusta(algoritmo, atributos, objetivo, solucionTemporal, n_exp, k)

            if rendimiento > mejorRendimiento:
                mejorRendimiento=rendimiento
                mejorSolucionTemporal=solucionTemporal

        solucionActual = mejorSolucionTemporal
        vectorMejoresSolucionesTemporales.append([mejorSolucionTemporal,mejorRendimiento,len(mejorSolucionTemporal)])

    return f.definirRetorno(vectorMejoresSolucionesTemporales).sort_values(by='Tamaño', ascending=True) 

In [4]:
def BusquedaSecuencialHaciaAtrasMixta(atributos, objetivo, algoritmo, n_exp, k, M):
    variablesDataset = np.array(atributos.columns[:].tolist())
    solucionActual = variablesDataset.copy()
    añadidos=[]
    eliminados=[]
    vectorMejoresSolucionesTemporales=[]
    contadorIteracionesSinAñadir=0
    mejorRendimientoIteracion=-1

    while contadorIteracionesSinAñadir < M:
        mejorRendimiento=-1
        mejorSolucionTemporal = []
        
        #Eliminar la peor variable
        peorVariableAEliminar=''
        for v in range(len(solucionActual)):
            if not solucionActual[v] in eliminados:
                solucionTemporal = solucionActual.copy();
                solucionTemporal = np.delete(solucionTemporal,v);
                rendimiento = f.EvaluacionRobusta(algoritmo, atributos, objetivo, solucionTemporal, n_exp, k)

                if rendimiento > mejorRendimiento:
                    mejorRendimiento=rendimiento
                    mejorSolucionTemporal=solucionTemporal
                    peorVariableAEliminar = solucionActual[v]

        if peorVariableAEliminar != '':
            solucionActual = mejorSolucionTemporal
            eliminados.append(peorVariableAEliminar)
        else:
            mejorRendimiento=mejorRendimientoIteracion

        #Añadir la mejor variable (solo si mejora el rendimiento)
        mejorVariableAñadir=''
        for v in range(len(variablesDataset)):
            if not np.any(solucionActual == variablesDataset[v]) and not variablesDataset[v] in añadidos:
                solucionTemporal = solucionActual.copy();
                solucionTemporal = np.append(solucionTemporal,variablesDataset[v])
                rendimiento = f.EvaluacionRobusta(algoritmo, atributos, objetivo, solucionTemporal, n_exp, k)

                if rendimiento > mejorRendimiento:
                    mejorRendimiento=rendimiento
                    mejorSolucionTemporal=solucionTemporal
                    mejorVariableAñadir=variablesDataset[v]

        if mejorVariableAñadir != '':
            solucionActual = mejorSolucionTemporal
            añadidos.append(mejorVariableAñadir)
            contadorIteracionesSinAñadir=0
        else:
            if len(eliminados) == (len(variablesDataset)):
                contadorIteracionesSinAñadir +=1

        mejorRendimientoIteracion = mejorRendimiento

        #Si la solución actual no está en el vector de mejores soluciones, la añadimos
        if not any(np.array_equal(sorted(solucionActual), sorted(sol[0])) for sol in vectorMejoresSolucionesTemporales):
            vectorMejoresSolucionesTemporales.append([solucionActual, mejorRendimiento, len(solucionActual)])

    return f.definirRetorno(vectorMejoresSolucionesTemporales)

# ***Aplicación en conjuntos de datos provistos***

#### Variables globales

In [43]:
n_exp = 10
k=7
M=10

In [100]:
hiperparametros_arbol = [
    {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2},
    {'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 4},
    {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2},
    {'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 2},
    {'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 4},
    {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 2}
]

In [101]:
hiperparametros_knn = [
    {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'},
    {'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'auto'},
    {'n_neighbors': 3, 'weights': 'distance', 'algorithm': 'auto'},
    {'n_neighbors': 10, 'weights': 'distance', 'algorithm': 'auto'},
    {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'ball_tree'},
    {'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'ball_tree'}
]

# *Titanic*

### Preprocesamiento de datos

In [39]:
atributos_t, objetivo_t, atributos_normalizados_t = f.preprocesamientoDeDatos('titanic.csv', 'Survived')

c:\Users\agutm\OneDrive\Escritorio\Estudio\Sistemasss\SEVILLA\InteligenciaArtificial\Trabajo\Trabajo\funciones.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.  0.  1.  0.  1.  1.  0.  1.  1.  0.5 1.  0.  1.  1.  1.  0.5 1.  0.5
 1.  1.  0.5 0.5 1.  0.  1.  1.  1.  0.  1.  1.  0.  0.  1.  0.5 0.  0.
 1.  1.  1.  1.  1.  0.5 1.  0.5 1.  1.  1.  1.  1.  1.  1.  1.  0.  0.5
 0.  0.  0.5 1.  0.5 1.  1.  0.  0.  1.  0.  1.  0.5 1.  1.  1.  0.5 1.
 0.5 1.  1.  1.  1.  1.  0.5 1.  1.  1.  1.  0.  0.5 1.  1.  1.  0.  1.
 1.  1.  0.  1.  1.  1.  0.  0.  0.5 0.5 1.  1.  0.  1.  1.  1.  1.  1.
 1.  1.  0.  1.  1.  1.  1.  1.  1.  0.5 0.  1.  0.5 1.  0.5 0.5 0.  1.
 1.  1.  1.  1.  1.  1.  1.  0.5 0.5 0.5 0.  0.  1.  0.  1.  1.  1.  1.
 0.5 0.5 1.  1.  0.5 0.5 0.5 0.  1.  1.  1.  0.  1.  1.  1.  1.  1.  0.5
 1.  1.  1.  1.  0.  1.  0.  1.  0.  1.  1.  1.  0.  1.  1.  0.  0.5 1.
 1.  0.5 1.  0.5 1.  0.  1.  0.  1.  1.  

In [40]:
atributos_normalizados_t.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Initial,Age_band,Family_Size,Alone,Fare_cat,Deck,Title,Is_Married
0,1.0,1.0,0.271174,0.125,0.0,0.014151,1.0,0.00,0.25,0.1,0.0,0.000000,1.00,0.6875,0
1,0.0,0.0,0.472229,0.125,0.0,0.139136,0.0,0.25,0.50,0.1,0.0,1.000000,0.25,0.7500,1
2,1.0,0.0,0.321438,0.000,0.0,0.015469,1.0,0.50,0.25,0.0,1.0,0.333333,1.00,0.5000,0
3,0.0,0.0,0.434531,0.125,0.0,0.103644,1.0,0.25,0.50,0.1,0.0,1.000000,0.25,0.7500,1
4,1.0,1.0,0.434531,0.000,0.0,0.015713,1.0,0.00,0.50,0.0,1.0,0.333333,1.00,0.6875,0


In [50]:
atributos_entrenamiento_t, atributos_prueba_t, objetivo_entrenamiento_t, objetivo_prueba_t = train_test_split(
                                                    atributos_normalizados_t, objetivo_t,
                                                    test_size=.2,
                                                    stratify=objetivo_t)

In [51]:
variablesDataset_t = np.array(atributos_t.columns[:].tolist())

### Ejemplos de aplicación de búsqueda secuencial hacia atrás

In [152]:
BusquedaSecuencialHaciaAtras(atributos_entrenamiento_t, objetivo_entrenamiento_t, DecisionTreeClassifier(), n_exp, k)

,Solución,Rendimiento,Tamaño
13,[Title],0.774978,1
12,"[SibSp, Title]",0.794046,2
11,"[SibSp, Fare_cat, Title]",0.776634,3
10,"[Pclass, SibSp, Fare_cat, Title]",0.780864,4
9,"[Pclass, SibSp, Age_band, Fare_cat, Title]",0.781628,5
8,"[Pclass, Sex, SibSp, Age_band, Fare_cat, Title]",0.796629,6
7,"[Pclass, Sex, SibSp, Age_band, Fare_cat, Deck, Title]",0.798561,7
6,"[Pclass, Sex, SibSp, Age_band, Alone, Fare_cat, Deck, Title]",0.797738,8
5,"[Pclass, Sex, SibSp, Age_band, Family_Size, Alone, Fare_cat, Deck, Title]",0.799045,9
4,"[Pclass, Sex, SibSp, Age_band, Family_Size, Alone, Fare_cat, Deck, Title, Is_Married]",0.791581,10


In [156]:
BusquedaSecuencialHaciaAtras(atributos_entrenamiento_t, objetivo_entrenamiento_t, KNeighborsClassifier(), n_exp, k)

,Solución,Rendimiento,Tamaño
13,[Initial],0.778062,1
12,"[Age, Initial]",0.758164,2
11,"[Age, Initial, Deck]",0.758996,3
10,"[Age, Initial, Fare_cat, Deck]",0.796362,4
9,"[Age, Initial, Fare_cat, Deck, Is_Married]",0.798455,5
8,"[Age, Initial, Family_Size, Fare_cat, Deck, Is_Married]",0.799528,6
7,"[Pclass, Age, Initial, Family_Size, Fare_cat, Deck, Is_Married]",0.808735,7
6,"[Pclass, Age, SibSp, Initial, Family_Size, Fare_cat, Deck, Is_Married]",0.814898,8
5,"[Pclass, Age, SibSp, Initial, Age_band, Family_Size, Fare_cat, Deck, Is_Married]",0.812662,9
4,"[Pclass, Age, SibSp, Parch, Initial, Age_band, Family_Size, Fare_cat, Deck, Is_Married]",0.813129,10


## Experimentos de búsqueda hacia atrás mixta con distintos hiperparámetros


### Árboles de decisión

#### Capacidad predictiva del conjunto completo

In [52]:
f.capacidadPredictivaDelConjuntoCompleto(DecisionTreeClassifier, hiperparametros_arbol, atributos_entrenamiento_t, objetivo_entrenamiento_t, variablesDataset_t, n_exp, k)

Hiperparámetros: {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}
Resultado: 0.8073397928236637
--------------------------------------------------
Hiperparámetros: {'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 4}
Resultado: 0.8037881984886592
--------------------------------------------------
Hiperparámetros: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2}
Resultado: 0.7517563849361084
--------------------------------------------------
Hiperparámetros: {'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 2}
Resultado: 0.802699988183859
--------------------------------------------------
Hiperparámetros: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 4}
Resultado: 0.7977369276908446
--------------------------------------------------
Hiperparámetros: {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 2}
Resultado: 0.749403706892186
--------------------------------------------------


#### Capacidad predictiva de diferentes subconjuntos de variables

In [53]:
dataframe_arbol_titanic=f.ExperimentosConHiperparametrosDeAlgoritmo(BusquedaSecuencialHaciaAtrasMixta, DecisionTreeClassifier, atributos_entrenamiento_t, objetivo_entrenamiento_t, hiperparametros_arbol, n_exp, k, M)

In [57]:
dataframe_arbol_titanic

,Solución,Rendimiento,Tamaño,Hiperparámetros
0,"[Pclass, Age, SibSp, Parch, Fare, Embarked, Initial, Age_band, Family_Size, Alone, Fare_cat, Title, Is_Married]",0.818967,13,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
1,"[Pclass, SibSp, Parch, Fare, Embarked, Initial, Age_band, Family_Size, Alone, Fare_cat, Title, Is_Married]",0.818967,12,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
2,"[Pclass, Parch, Fare, Embarked, Initial, Age_band, Family_Size, Alone, Fare_cat, Title, Is_Married]",0.818967,11,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
3,"[Pclass, Fare, Embarked, Initial, Age_band, Family_Size, Alone, Fare_cat, Title, Is_Married]",0.818967,10,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
4,"[Pclass, Fare, Initial, Age_band, Family_Size, Alone, Fare_cat, Title, Is_Married]",0.818967,9,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
5,"[Pclass, Fare, Initial, Family_Size, Alone, Fare_cat, Title, Is_Married]",0.818967,8,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
6,"[Pclass, Fare, Initial, Family_Size, Fare_cat, Title, Is_Married]",0.818967,7,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
7,"[Pclass, Fare, Initial, Family_Size, Title, Is_Married]",0.818967,6,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
8,"[Pclass, Fare, Initial, Family_Size, Is_Married]",0.818967,5,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
9,"[Pclass, Fare, Initial, Family_Size]",0.818967,4,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"


### Entrenamiento con los mejores hiperparámetros y conjunto de características

In [138]:
mejor_solucion_arbol_t, mejores_hiperparametros_arbol_t = f.obtener_mejor_solucion(dataframe_arbol_titanic)
print("Mejor solución: ", mejor_solucion_arbol_t)
print("Mejores hiperparámetros: ", mejores_hiperparametros_arbol_t)

hiperparametros_arbol_dict_t= ast.literal_eval(mejores_hiperparametros_arbol_t)
arbol_t= DecisionTreeClassifier(**hiperparametros_arbol_dict_t)
arbol_t.fit(atributos_entrenamiento_t[mejor_solucion_arbol_t], objetivo_entrenamiento_t)

Mejor solución:  ['Pclass' 'Fare' 'Initial']
Mejores hiperparámetros:  {'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 2}


DecisionTreeClassifier(criterion='entropy', max_depth=3)

### Evaluación de la capacidad de generalización

In [130]:
predicciones_arbol_t= arbol_t.predict(atributos_prueba_t[mejor_solucion_arbol_t])
confusion_matrix(objetivo_prueba_t, predicciones_arbol_t)

array([[97, 13],
       [20, 49]], dtype=int64)

In [131]:
recall_score(objetivo_prueba_t, predicciones_arbol_t)

0.7101449275362319

### KNN

#### Capacidad predictiva del conjunto completo

In [83]:
f.capacidadPredictivaDelConjuntoCompleto(KNeighborsClassifier, hiperparametros_knn, atributos_entrenamiento_t, objetivo_entrenamiento_t, variablesDataset_t, n_exp, k)

Hiperparámetros: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'}
Resultado: 0.7912284847768716
--------------------------------------------------
Hiperparámetros: {'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'auto'}
Resultado: 0.7749601908588082
--------------------------------------------------
Hiperparámetros: {'n_neighbors': 3, 'weights': 'distance', 'algorithm': 'auto'}
Resultado: 0.7786653950248421
--------------------------------------------------
Hiperparámetros: {'n_neighbors': 10, 'weights': 'distance', 'algorithm': 'auto'}
Resultado: 0.7816855443583552
--------------------------------------------------
Hiperparámetros: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'ball_tree'}
Resultado: 0.7912284847768716
--------------------------------------------------
Hiperparámetros: {'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'ball_tree'}
Resultado: 0.7749601908588082
--------------------------------------------------


#### Capacidad predictiva de diferentes subconjuntos de variables

In [81]:
dataframe_knn_titanic=f.ExperimentosConHiperparametrosDeAlgoritmo(BusquedaSecuencialHaciaAtrasMixta, KNeighborsClassifier, atributos_entrenamiento_t, objetivo_entrenamiento_t, hiperparametros_knn, n_exp, k, M)

In [82]:
dataframe_knn_titanic

,Solución,Rendimiento,Tamaño,Hiperparámetros
0,"[Pclass, Sex, Age, SibSp, Fare, Initial, Age_band, Family_Size, Alone, Fare_cat, Deck, Is_Married]",0.825512,12,"{'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'}"
1,"[Pclass, Sex, Age, SibSp, Parch, Fare, Initial, Age_band, Family_Size, Alone, Fare_cat, Deck, Is_Married]",0.823757,13,"{'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'}"
2,"[Pclass, Sex, Age, SibSp, Parch, Fare, Initial, Age_band, Family_Size, Alone, Fare_cat, Deck, Title, Is_Married]",0.816660,14,"{'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'}"
3,"[Pclass, Sex, Age, SibSp, Parch, Fare, Embarked, Initial, Age_band, Family_Size, Alone, Fare_cat]",0.805555,12,"{'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'auto'}"
4,"[Pclass, Sex, Age, SibSp, Parch, Fare, Embarked, Initial, Age_band, Family_Size, Alone, Fare_cat, Title]",0.805151,13,"{'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'auto'}"
5,"[Pclass, Sex, Age, SibSp, Parch, Fare, Embarked, Initial, Age_band, Family_Size, Alone, Fare_cat, Title, Is_Married]",0.803873,14,"{'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'auto'}"
6,"[Pclass, Sex, Age, SibSp, Initial, Age_band, Fare_cat, Deck, Is_Married]",0.810878,9,"{'n_neighbors': 3, 'weights': 'distance', 'algorithm': 'auto'}"
7,"[Pclass, Sex, Age, SibSp, Initial, Age_band, Fare_cat, Deck]",0.810878,8,"{'n_neighbors': 3, 'weights': 'distance', 'algorithm': 'auto'}"
8,"[Pclass, Sex, Age, SibSp, Initial, Age_band, Family_Size, Fare_cat, Deck, Is_Married]",0.805418,10,"{'n_neighbors': 3, 'weights': 'distance', 'algorithm': 'auto'}"
9,"[Pclass, Sex, Age, SibSp, Initial, Age_band, Family_Size, Alone, Fare_cat, Deck, Is_Married]",0.803057,11,"{'n_neighbors': 3, 'weights': 'distance', 'algorithm': 'auto'}"


### Entrenamiento con los mejores hiperparámetros y conjunto de características

In [139]:
mejor_solucion_knn_t, mejores_hiperparametros_knn_t = f.obtener_mejor_solucion(dataframe_knn_titanic)
print("Mejor solución: ", mejor_solucion_knn_t)
print("Mejores hiperparámetros: ", mejores_hiperparametros_knn_t)

hiperparametros_knn_dict_t= ast.literal_eval(mejores_hiperparametros_knn_t)
knn_t= KNeighborsClassifier(**hiperparametros_knn_dict_t)
knn_t.fit(atributos_entrenamiento_t[mejor_solucion_knn_t], objetivo_entrenamiento_t)

Mejor solución:  ['Pclass' 'Sex' 'Age' 'SibSp' 'Fare' 'Initial' 'Age_band' 'Family_Size'
 'Alone' 'Fare_cat' 'Deck' 'Is_Married']
Mejores hiperparámetros:  {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'}


KNeighborsClassifier(n_neighbors=3)

### Evaluación de la capacidad de generalización

In [134]:
predicciones_knn_t= knn_t.predict(atributos_prueba_t[mejor_solucion_knn_t])
confusion_matrix(objetivo_prueba_t, predicciones_knn_t)

array([[95, 15],
       [18, 51]], dtype=int64)

In [135]:
recall_score(objetivo_prueba_t, predicciones_knn_t)

0.7391304347826086


# *Breast Cancer*

### Preprocesamiento de datos

In [106]:
atributos_b, objetivo_b, atributos_normalizados_b = f.preprocesamientoDeDatos('BreastCancer.csv', 'diagnosis')

In [107]:
atributos_normalizados_b.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,0.356147,0.120469,0.369034,0.273811,0.159296,0.351398,0.135682,0.300625,0.311645,0.183042,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,0.156437,0.082589,0.124440,0.125660,0.119387,0.081323,0.046970,0.253836,0.084539,0.091110,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
2,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,0.229622,0.094303,0.180370,0.162922,0.150831,0.283955,0.096768,0.389847,0.205690,0.127006,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
3,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,0.139091,0.175875,0.126655,0.038155,0.251453,0.543215,0.142955,0.353665,0.728148,0.287205,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
4,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,0.233822,0.093065,0.220563,0.163688,0.332359,0.167918,0.143636,0.357075,0.136179,0.145800,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595


In [108]:
atributos_entrenamiento_b, atributos_prueba_b, objetivo_entrenamiento_b, objetivo_prueba_b = train_test_split(
                                                    atributos_normalizados_b, objetivo_b,
                                                    test_size=.2,
                                                    stratify=objetivo_b)

In [109]:
variablesDataset_b = np.array(atributos_b.columns[:].tolist())

### Ejemplos de aplicación de búsqueda secuencial hacia atrás

In [154]:
BusquedaSecuencialHaciaAtras(atributos_entrenamiento_b, objetivo_entrenamiento_b, DecisionTreeClassifier(), n_exp, k)

,Solución,Rendimiento,Tamaño
28,[worst radius],0.861444,1
27,"[worst radius, worst texture]",0.913398,2
26,"[mean concave points, worst radius, worst texture]",0.936614,3
25,"[mean concave points, mean fractal dimension, worst radius, worst texture]",0.944799,4
24,"[mean concave points, mean fractal dimension, area error, worst radius, worst texture]",0.955596,5
23,"[mean concave points, mean fractal dimension, perimeter error, area error, worst radius, worst texture]",0.954951,6
22,"[mean compactness, mean concave points, mean fractal dimension, perimeter error, area error, worst radius, worst texture]",0.955079,7
21,"[mean compactness, mean concave points, mean fractal dimension, perimeter error, area error, worst radius, worst texture, worst symmetry]",0.955842,8
20,"[mean area, mean compactness, mean concave points, mean fractal dimension, perimeter error, area error, worst radius, worst texture, worst symmetry]",0.951255,9
19,"[mean area, mean compactness, mean concave points, mean fractal dimension, perimeter error, area error, worst radius, worst texture, worst concavity, worst symmetry]",0.956572,10


In [155]:
BusquedaSecuencialHaciaAtras(atributos_entrenamiento_b, objetivo_entrenamiento_b, KNeighborsClassifier(), n_exp, k)

,Solución,Rendimiento,Tamaño
28,[worst perimeter],0.894217,1
27,"[worst texture, worst perimeter]",0.934294,2
26,"[mean concave points, worst texture, worst perimeter]",0.958101,3
25,"[mean concave points, mean symmetry, worst texture, worst perimeter]",0.961197,4
24,"[mean concave points, mean symmetry, concave points error, worst texture, worst perimeter]",0.963921,5
23,"[mean concave points, mean symmetry, concave points error, worst texture, worst perimeter, worst concavity]",0.961170,6
22,"[mean area, mean concave points, mean symmetry, concave points error, worst texture, worst perimeter, worst concavity]",0.967411,7
21,"[mean area, mean concave points, mean symmetry, texture error, concave points error, worst texture, worst perimeter, worst concavity]",0.970359,8
20,"[mean area, mean concave points, mean symmetry, texture error, concave points error, worst texture, worst perimeter, worst area, worst concavity]",0.975333,9
19,"[mean area, mean concave points, mean symmetry, texture error, smoothness error, concave points error, worst texture, worst perimeter, worst area, worst concavity]",0.967774,10


## Experimentos de búsqueda hacia atrás mixta con distintos hiperparámetros

### Árboles de decisión

#### Capacidad predictiva del conjunto completo

In [110]:
f.capacidadPredictivaDelConjuntoCompleto(DecisionTreeClassifier, hiperparametros_arbol, atributos_entrenamiento_b, objetivo_entrenamiento_b, variablesDataset_b, n_exp, k)

Hiperparámetros: {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}
Resultado: 0.9301117886178863
--------------------------------------------------
Hiperparámetros: {'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 4}
Resultado: 0.9386146922183508
--------------------------------------------------
Hiperparámetros: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2}
Resultado: 0.938927700348432
--------------------------------------------------
Hiperparámetros: {'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 2}
Resultado: 0.9335307781649245
--------------------------------------------------
Hiperparámetros: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 4}
Resultado: 0.9352700348432055
--------------------------------------------------
Hiperparámetros: {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 2}
Resultado: 0.936824912891986
--------------------------------------------------


#### Capacidad predictiva de diferentes subconjuntos de variables

In [112]:
dataframe_arbol_breastCancer=f.ExperimentosConHiperparametrosDeAlgoritmo(BusquedaSecuencialHaciaAtrasMixta, DecisionTreeClassifier, atributos_entrenamiento_b, objetivo_entrenamiento_b, hiperparametros_arbol, n_exp, k, M)

In [122]:
dataframe_arbol_breastCancer

,Solución,Rendimiento,Tamaño,Hiperparámetros
0,"[mean texture, mean concave points, area error, compactness error, concavity error, worst radius, worst texture, worst symmetry, mean area, mean compactness, symmetry error]",0.966854,11,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
1,"[mean texture, mean compactness, mean concave points, mean symmetry, area error, compactness error, concavity error, concave points error, fractal dimension error, worst radius, worst texture, worst symmetry, mean area]",0.966739,13,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
2,"[mean texture, mean compactness, mean concave points, mean symmetry, area error, smoothness error, compactness error, concavity error, concave points error, symmetry error, fractal dimension error, worst radius, worst texture, worst fractal dimension, worst symmetry, mean area]",0.966552,16,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
3,"[mean texture, mean compactness, mean concave points, mean symmetry, area error, compactness error, concavity error, concave points error, symmetry error, fractal dimension error, worst radius, worst texture, worst fractal dimension, worst symmetry, mean area]",0.966510,15,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
4,"[worst symmetry, mean area, mean compactness, symmetry error, mean perimeter, mean symmetry, mean texture, area error, worst texture, worst radius, mean concave points, compactness error, concavity error, fractal dimension error]",0.966318,14,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
5,"[mean texture, mean concave points, area error, compactness error, concavity error, concave points error, fractal dimension error, worst radius, worst texture, worst symmetry, mean area, mean compactness]",0.965957,12,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
6,"[mean texture, mean compactness, mean concave points, mean symmetry, area error, compactness error, concavity error, concave points error, symmetry error, fractal dimension error, worst radius, worst texture, worst symmetry, mean area]",0.965897,14,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
7,"[mean texture, mean concave points, area error, compactness error, concavity error, fractal dimension error, worst radius, worst texture, worst symmetry, mean area, mean compactness, symmetry error]",0.965774,12,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
8,"[mean texture, mean perimeter, mean compactness, mean concave points, mean symmetry, perimeter error, area error, smoothness error, compactness error, concavity error, concave points error, symmetry error, fractal dimension error, worst radius, worst texture, worst concavity, worst fractal dimension, worst symmetry]",0.965659,18,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"
9,"[mean texture, mean concave points, area error, compactness error, worst radius, worst texture, worst symmetry, mean area, mean compactness, symmetry error, mean perimeter]",0.965659,11,"{'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}"


### Entrenamiento con los mejores hiperparámetros y conjunto de características

In [143]:
mejor_solucion_arbol_b, mejores_hiperparametros_arbol_b = f.obtener_mejor_solucion(dataframe_arbol_breastCancer)
print("Mejor solución: ", mejor_solucion_arbol_b)
print("Mejores hiperparámetros: ", mejores_hiperparametros_arbol_b)

hiperparametros_arbol_dict_b= ast.literal_eval(mejores_hiperparametros_arbol_b)
arbol_b= DecisionTreeClassifier(**hiperparametros_arbol_dict_b)
arbol_b.fit(atributos_entrenamiento_b[mejor_solucion_arbol_b], objetivo_entrenamiento_b)

Mejor solución:  ['mean texture' 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'symmetry error' 'worst texture' 'worst perimeter'
 'worst area' 'worst compactness' 'mean fractal dimension'
 'mean perimeter']
Mejores hiperparámetros:  {'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 2}


DecisionTreeClassifier(criterion='entropy', max_depth=3)

### Evaluación de la capacidad de generalización

In [141]:
predicciones_arbol_b= arbol_b.predict(atributos_prueba_b[mejor_solucion_arbol_b])
confusion_matrix(objetivo_prueba_b, predicciones_arbol_b)

array([[37,  5],
       [ 1, 71]], dtype=int64)

In [142]:
recall_score(objetivo_prueba_b, predicciones_arbol_b)

0.9861111111111112

## kNN

#### Capacidad predictiva del conjunto completo

In [144]:
f.capacidadPredictivaDelConjuntoCompleto(KNeighborsClassifier, hiperparametros_knn, atributos_entrenamiento_b, objetivo_entrenamiento_b, variablesDataset_b, n_exp, k)

Hiperparámetros: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'}
Resultado: 0.9572415795586527
--------------------------------------------------
Hiperparámetros: {'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'auto'}
Resultado: 0.9742973286875726
--------------------------------------------------
Hiperparámetros: {'n_neighbors': 3, 'weights': 'distance', 'algorithm': 'auto'}
Resultado: 0.9572415795586527
--------------------------------------------------
Hiperparámetros: {'n_neighbors': 10, 'weights': 'distance', 'algorithm': 'auto'}
Resultado: 0.9672299651567944
--------------------------------------------------
Hiperparámetros: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'ball_tree'}
Resultado: 0.9572415795586527
--------------------------------------------------
Hiperparámetros: {'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'ball_tree'}
Resultado: 0.9742973286875726
--------------------------------------------------


#### Capacidad predictiva de diferentes subconjuntos de variables

In [137]:
dataframe_knn_breastCancer= f.ExperimentosConHiperparametrosDeAlgoritmo(BusquedaSecuencialHaciaAtrasMixta, KNeighborsClassifier, atributos_entrenamiento_b, objetivo_entrenamiento_b, hiperparametros_knn, n_exp, k, M)

In [145]:
dataframe_knn_breastCancer

,Solución,Rendimiento,Tamaño,Hiperparámetros
0,"[mean texture, mean perimeter, mean area, mean smoothness, mean concavity, mean concave points, mean fractal dimension, radius error, texture error, perimeter error, area error, smoothness error, compactness error, concavity error, concave points error, symmetry error, fractal dimension error, worst radius, worst texture, worst perimeter, worst area, worst compactness, worst concavity, worst symmetry]",0.979016,24,"{'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'}"
1,"[mean texture, mean perimeter, mean area, mean smoothness, mean concavity, mean concave points, mean fractal dimension, radius error, texture error, perimeter error, area error, smoothness error, compactness error, concavity error, concave points error, symmetry error, worst radius, worst texture, worst perimeter, worst area, worst compactness, worst concavity, worst symmetry]",0.979016,23,"{'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'}"
2,"[mean texture, mean perimeter, mean area, mean smoothness, mean compactness, mean concavity, mean concave points, mean fractal dimension, radius error, texture error, perimeter error, area error, smoothness error, compactness error, concavity error, concave points error, symmetry error, fractal dimension error, worst radius, worst texture, worst perimeter, worst area, worst compactness, worst concavity, worst symmetry]",0.974297,25,"{'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'}"
3,"[mean texture, mean perimeter, mean area, mean smoothness, mean compactness, mean concavity, mean concave points, mean symmetry, mean fractal dimension, radius error, texture error, perimeter error, area error, smoothness error, compactness error, concavity error, concave points error, symmetry error, fractal dimension error, worst radius, worst texture, worst perimeter, worst area, worst compactness, worst concavity, worst symmetry]",0.971960,26,"{'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'}"
4,"[mean texture, mean perimeter, mean area, mean smoothness, mean compactness, mean concavity, mean concave points, mean symmetry, mean fractal dimension, radius error, texture error, perimeter error, area error, smoothness error, compactness error, concavity error, concave points error, symmetry error, fractal dimension error, worst radius, worst texture, worst perimeter, worst area, worst compactness, worst concavity, worst concave points, worst symmetry]",0.967793,27,"{'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'}"
5,"[mean texture, mean perimeter, mean area, mean smoothness, mean compactness, mean concavity, mean concave points, mean symmetry, mean fractal dimension, radius error, texture error, perimeter error, area error, smoothness error, compactness error, concavity error, concave points error, symmetry error, fractal dimension error, worst radius, worst texture, worst perimeter, worst area, worst compactness, worst concavity, worst concave points, worst symmetry, worst fractal dimension]",0.966051,28,"{'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'}"
6,"[mean texture, mean perimeter, mean area, mean smoothness, mean compactness, mean concavity, mean concave points, mean symmetry, mean fractal dimension, radius error, texture error, perimeter error, area error, smoothness error, compactness error, concavity error, concave points error, symmetry error, fractal dimension error, worst radius, worst texture, worst perimeter, worst area, worst smoothness, worst compactness, worst concavity, worst concave points, worst symmetry, worst fractal dimension]",0.963075,29,"{'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'}"
7,"[mean radius, mean texture, mean perimeter, mean area, mean smoothness, mean compactness, mean concavity, mean concave points, texture error, perimeter error, area error, smoothness error, compactness error, concavity error, concave points error, symmetry error, fractal dimension error, worst radi

### Entrenamiento con los mejores hiperparámetros y conjunto de características

In [146]:
mejor_solucion_knn_b, mejores_hiperparametros_knn_b = f.obtener_mejor_solucion(dataframe_knn_breastCancer)
print("Mejor solución: ", mejor_solucion_knn_b)
print("Mejores hiperparámetros: ", mejores_hiperparametros_knn_b)

hiperparametros_knn_dict_b= ast.literal_eval(mejores_hiperparametros_knn_b)
knn_b= KNeighborsClassifier(**hiperparametros_knn_dict_b)
knn_b.fit(atributos_entrenamiento_b[mejor_solucion_knn_b], objetivo_entrenamiento_b)

Mejor solución:  ['mean texture' 'mean perimeter' 'mean smoothness' 'mean concave points'
 'mean symmetry' 'texture error' 'smoothness error' 'compactness error'
 'concave points error' 'fractal dimension error' 'worst radius'
 'worst texture' 'worst perimeter' 'worst area' 'worst concavity'
 'worst concave points' 'worst fractal dimension']
Mejores hiperparámetros:  {'n_neighbors': 10, 'weights': 'distance', 'algorithm': 'auto'}


KNeighborsClassifier(n_neighbors=10, weights='distance')

### Evaluación de la capacidad de generalización

In [147]:
predicciones_knn_b= knn_b.predict(atributos_prueba_b[mejor_solucion_knn_b])
confusion_matrix(objetivo_prueba_b, predicciones_knn_b)

array([[39,  3],
       [ 1, 71]], dtype=int64)

In [148]:
recall_score(objetivo_prueba_b, predicciones_knn_b)

0.9861111111111112